## Environment Setup

In [ ]:
import dotenv

dotenv.load_dotenv()


True

## Imports Stuff

In [ ]:
from datetime import datetime
import hopsworks
import pandas as pd
from xgboost import XGBClassifier


## Connect to Hopsworks and Load Data

In [ ]:
project = hopsworks.login(project="ATP_Tennis_Prediction")
fs = project.get_feature_store()

feature_view = fs.get_feature_view(
    name="tennis_match_prediction",
    version=1,
)

2026-01-03 14:31:47,165 INFO: Initializing external client
2026-01-03 14:31:47,166 INFO: Base URL: https://eu-west.cloud.hopsworks.ai:443
2026-01-03 14:31:48,547 INFO: Python Engine initialized.

Logged in to project, explore it here https://eu-west.cloud.hopsworks.ai:443/p/3204


In [ ]:
mr = project.get_model_registry()

retrieved_model = mr.get_model(
    name="tennis_match_predictor",  # ← Your model name
    version=1,
)

saved_model_dir = retrieved_model.download()

model = XGBClassifier()
model.load_model(saved_model_dir + "/model.json")


Downloading: 0.000%|          | 0/597627 elapsed<00:00 remaining<?

### Holdout
currently no new matches happening yet until January

In [ ]:
# Get ALL matches from main feature group
tennis_fg = fs.get_feature_group(
    name="tennis_matches",
    version=2,
)

all_matches = tennis_fg.read()

# Filter for "upcoming" (pretend 2025 hasn't happened)
all_matches["date"] = pd.to_datetime(all_matches["date"])
df_new = all_matches[all_matches["date"].dt.year == 2025].copy()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (20.87s) 


### Upcoming Matches 

In [ ]:
# upcoming_fg = fs.get_feature_group(
#     name="tennis_upcoming_matches",
#     version=1,
# )

# df_new = upcoming_fg.read()


## Update Model 

In [ ]:
drop_list = [
    # Metadata
    "date",
    "player_1",
    "player_2",
    "winner",
    "timestamp",
    # Categorical text (have encoded versions)
    "tournament",
    "surface",
    "series",
    "round",
    "court",
    "tournament_clean",
]

# Also drop target (we're predicting it!)
drop_list.append("player_1_won")

# Filter for columns that exist
cols_to_drop = [col for col in drop_list if col in df_new.columns]

# Create features
features = df_new.drop(cols_to_drop, axis=1)

print(f"Features shape: {features.shape}")
print(f"Features columns: {features.columns.tolist()}")

# Verify no missing values
if features.isnull().sum().sum() > 0:
    print("Missing values found!")
    features = features.fillna(0)


Features shape: (19904, 48)
Features columns: ['best_of', 'rank_1', 'rank_2', 'rank_diff', 'pts_1', 'pts_2', 'pts_diff', 'odd_1', 'odd_2', 'odds_diff', 'p1_total_matches', 'p1_wins', 'p1_losses', 'p2_total_matches', 'p2_wins', 'p2_losses', 'p1_win_pct', 'p2_win_pct', 'p1_winpct_hard', 'p2_winpct_hard', 'p1_winpct_clay', 'p2_winpct_clay', 'p1_winpct_grass', 'p2_winpct_grass', 'p1_winpct_carpet', 'p2_winpct_carpet', 'p1_winpct_atp250', 'p2_winpct_atp250', 'p1_winpct_atp500', 'p2_winpct_atp500', 'p1_winpct_grand_slam', 'p2_winpct_grand_slam', 'p1_winpct_masters_1000', 'p2_winpct_masters_1000', 'p1_winpct_indoor', 'p2_winpct_indoor', 'p1_winpct_outdoor', 'p2_winpct_outdoor', 'tournament_encoded', 'surface_encoded', 'series_encoded', 'round_encoded', 'court_encoded', 'win_pct_diff', 'experience_diff', 'surface_advantage', 'series_advantage', 'court_advantage']


## Prediction

In [ ]:
# Predict match outcomes
predictions = model.predict(features)

# Get probabilities
prediction_probabilities = model.predict_proba(features)
prob_player1_wins = prediction_probabilities[:, 1]  # P(Player 1 wins)

print(f"Predictions made: {len(predictions)}")
print(f"Player 1 wins predicted: {predictions.sum()} ({predictions.mean():.1%})")


Predictions made: 19904
Player 1 wins predicted: 9980 (50.1%)


In [ ]:
results = df_new[["date", "player_1", "player_2"]].copy()

# Add predictions
results["predicted_player_1_wins"] = predictions
results["player_1_win_probability"] = prob_player1_wins

results["predicted_winner"] = results.apply(
    lambda row: row["player_1"]
    if row["predicted_player_1_wins"] == 1
    else row["player_2"],
    axis=1,
)

results["confidence"] = [x if x > 0.5 else 1 - x for x in prob_player1_wins]

# Validation
if "player_1_won" in df_new.columns:
    results["actual_winner"] = df_new.apply(
        lambda row: row["player_1"] if row["player_1_won"] == 1 else row["player_2"],
        axis=1,
    )
    results["correct"] = results["predicted_winner"] == results["actual_winner"]

    accuracy = results["correct"].mean()
    print(f"Validation Accuracy: {accuracy:.2%}")

results.head(20)

Validation Accuracy: 69.27%


,date,player_1,player_2,predicted_player_1_wins,player_1_win_probability,predicted_winner,confidence,actual_winner,correct
129557,2025-01-01,Bergs Z.,Fils A.,0,0.342426,Fils A.,0.657574,Fils A.,True
129558,2025-01-01,Bergs Z.,Fils A.,0,0.342426,Fils A.,0.657574,Fils A.,True
129559,2025-01-01,Bonzi B.,Jarry N.,1,0.705217,Bonzi B.,0.705217,Jarry N.,False
129560,2025-01-01,Bonzi B.,Jarry N.,1,0.705217,Bonzi B.,0.705217,Jarry N.,False
129561,2025-01-01,Dimitrov G.,Vukic A.,1,0.818721,Dimitrov G.,0.818721,Dimitrov G.,True
129562,2025-01-01,Dimitrov G.,Vukic A.,1,0.818721,Dimitrov G.,0.818721,Dimitrov G.,True
129563,2025-01-01,Fils A.,Bergs Z.,1,0.658944,Fils A.,0.658944,Fils A.,True
129564,2025-01-01,Jarry N.,Bonzi B.,0,0.317755,Bonzi B.,0.682245,Jarry N.,False
129565,2025-01-01,Kecmanovic M.,Muller A.,1,0.690048,Kecmanovic M.,0.690048,Muller A.,False
129566,2025-01-01,Khachanov K.,Nishikori K.,1,0.513054,Khachanov K.,0.513054,Nishikori K.,False


In [ ]:
results.to_csv("tennis_predictions.csv", index=False, sep=";")
print(f"Saved {len(results)} predictions to tennis_predictions.csv")

Saved 19904 predictions to tennis_predictions.csv
